In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime


In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
year = 2010 # Year we're currently testing.
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_' + str(year),cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 2010 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XgWZHQpAAEYAAIfWb9YAAACJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_2010","wgTitle":"List of Billboard Hot 100 top-ten singles in 2010","wgCurRevisionId":899254653,"wgRevisionId":899254653,"wgArticleId":25552100,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [10]:
my_table = soup.find('table',{'class':'wikitable sortable'})
type(my_table)

bs4.element.Tag

In [34]:
tableheaderincluded = True
swiftcount = 0

# The following code mines the first table found on the Wikipedia page and extracts only table elements that do not correspond to a month and date. We don't care about the exact month and day that the song achieved Top 10 glory or their peek date. Furthermore, the way Wikipedia's tables are formatted, there are occasionally multiple songs that were all entered on the same day, and that date is listed only once, which creates problems for our script later down the road.

In [37]:
Titles = []
try:
    tds = my_table.findAll('td')
except AttributeError:
    my_table = soup.find('table',{'class':'wikitable'})
    tableheaderincluded = False
    tds = my_table.findAll('td')

for td in tds:
    try:
        d = datetime.datetime.strptime(td.get_text()[:-1], "%B %d") # Check if the current string is a Month Day format.
    except ValueError:
        try:
            d = datetime.datetime.strptime(td.get_text()[:td.get_text().rfind('(')-1], "%B %d") # Catch a rare case where a song from the previous year charted again.
        except ValueError:
            try:
                d = datetime.datetime.strptime(td.get_text()[:td.get_text().rfind('[')-1], "%B %d") # Catch a rare case where a song's top ten entry date has an annotation.
            except ValueError:
                try:
                    if td.attrs['rowspan']:
                        taylorswift = td.get_text()[:-1]
                        swiftcount = 5
                except KeyError:
                    pass
                if td.get_text()[0] != '[' and td.get_text()[0] and td.get_text()[0].strip(): # Drop annotations column.
                    if swiftcount == 1:
                        print(taylorswift + ' appended!')
                        Titles.append(taylorswift)
                        taylorswift = ''
                        swiftcount = 0
                    if swiftcount > 0:
                        swiftcount = swiftcount-1;
                    Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.

Taylor Swift appended!


In [38]:
print(Titles)

['"Replay"', 'Iyaz', '2', '14', '"Need You Now" (#2)', 'Lady Antebellum', '2', '16', '"Tik Tok" (#1)', 'Kesha', '1', '20', '"Sexy Bitch"', 'David Guetta featuring Akon', '5', '13', '"Do You Remember"', 'Jay Sean featuring Sean Paul and Lil Jon', '10', '1', '"Blah Blah Blah" ◁', 'Kesha featuring 3OH!3', '7', '1', '"BedRock"', 'Young Money featuring Lloyd', '2', '13', '"Hard"', 'Rihanna featuring Young Jeezy', '8', '4', '"Hey, Soul Sister" (#3) [A]', 'Train', '3', '19', '"How Low"', 'Ludacris', '6', '7', '"Baby" ◁', 'Justin Bieber featuring Ludacris', '5', '5', '"Today Was a Fairytale" ◁', 'Taylor Swift', '2', '1', '"Imma Be"', 'The Black Eyed Peas', '1', '11', '"We Are the World 25 for Haiti" ◁', 'Artists for Haiti', '2', '2', '"In My Head" [B]', 'Jason Derulo', '5', '10', '"Say Aah"', 'Trey Songz featuring Fabolous', '9', '2', '"Rude Boy"', 'Rihanna', '1', '13', '"Break Your Heart" (#10)', 'Taio Cruz featuring Ludacris', '1', '16', '"Nothin\' on You"', 'B.o.B featuring Bruno Mars', '1'

In [39]:
nptitles = np.asarray(Titles).copy()

In [40]:
len(nptitles)

236

In [41]:
numsongs = int(np.prod(nptitles.shape)/4) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,4) # Reshape the array based on the number of songs and the number of columns.

In [42]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Title','Artist','Peak','Number of weeks in top ten']
songdata['Year'] = str(year)
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
cleanweeks = []
for i in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
    try:                                         # problem in the current year.
        cleanweeks.append(int(i[:i.rindex('*')]))
    except ValueError:
        cleanweeks.append(int(i))

songdata['Number of weeks in top ten'] = cleanweeks
del songdata['Peak'] # This will not be relevant to our analysis.

In [44]:
songdata

,Title,Artist,Number of weeks in top ten,Year
0,Replay,Iyaz,14,2010
1,Need You Now,Lady Antebellum,16,2010
2,Tik Tok,Kesha,20,2010
3,Sexy Bitch,David Guetta featuring Akon,13,2010
4,Do You Remember,Jay Sean featuring Sean Paul and Lil Jon,1,2010
5,Blah Blah Blah,Kesha featuring 3OH!3,1,2010
6,BedRock,Young Money featuring Lloyd,13,2010
7,Hard,Rihanna featuring Young Jeezy,4,2010
8,"Hey, Soul Sister",Train,19,2010
9,How Low,Ludacris,7,2010


In [43]:
songdata.to_csv(path_or_buf='data/' + str(year) + 'top10songs.csv',index=False) # Export our sample data table.